In [25]:
import keras
from keras import models
from keras import layers
from keras import optimizers 
from keras.preprocessing.image import ImageDataGenerator
import os

In [26]:
image_size = 224
trainval_or_classes = 1
image_dir = '/data/ml_data/images/oilwater'
train_dir = ''
validation_dir = ''

num_classes = 0
for name in os.listdir(image_dir):
        if(os.path.isdir(os.path.join(image_dir, name))):
            num_classes+=1
print("num_classes = " + str(num_classes))

vgg_conv = keras.applications.vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

# Freeze the layers except the last 4 layers
for layer in vgg_conv.layers[:-4]:
    layer.trainable = False
 
# Check the trainable status of the individual layers
for layer in vgg_conv.layers:
    print(layer, layer.trainable)

num_classes = 2
<keras.engine.input_layer.InputLayer object at 0x7f9cfa509860> False
<keras.layers.convolutional.Conv2D object at 0x7f9cf8d3cb70> False
<keras.layers.convolutional.Conv2D object at 0x7f9cf8cdfc88> False
<keras.layers.pooling.MaxPooling2D object at 0x7f9cf8ccae48> False
<keras.layers.convolutional.Conv2D object at 0x7f9cf8cf0710> False
<keras.layers.convolutional.Conv2D object at 0x7f9cf9520400> False
<keras.layers.pooling.MaxPooling2D object at 0x7f9cf953b400> False
<keras.layers.convolutional.Conv2D object at 0x7f9cf953b908> False
<keras.layers.convolutional.Conv2D object at 0x7f9cf9555630> False
<keras.layers.convolutional.Conv2D object at 0x7f9cf94ee1d0> False
<keras.layers.pooling.MaxPooling2D object at 0x7f9cf94a2710> False
<keras.layers.convolutional.Conv2D object at 0x7f9cf94a28d0> False
<keras.layers.convolutional.Conv2D object at 0x7f9cf94bd5f8> False
<keras.layers.convolutional.Conv2D object at 0x7f9cf94d6198> False
<keras.layers.pooling.MaxPooling2D object at

In [27]:
# Create the model
model = models.Sequential()
 
# Add the vgg convolutional base model
model.add(vgg_conv)
 
# Add new layers
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))

num_classes=1

model.add(layers.Dense(num_classes, activation='softmax'))
 
# Show a summary of the model. Check the number of trainable parameters
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_5 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_9 (Dense)              (None, 1024)              25691136  
_________________________________________________________________
dropout_5 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 1025      
Total params: 40,406,849
Trainable params: 32,771,585
Non-trainable params: 7,635,264
_________________________________________________________________


In [28]:
train_batchsize = 32
val_batchsize = 16

valid = True
if(trainval_or_classes == 0):
    if(train_dir or validation_dir == ''):
        print("train_dir and validation_dir must be specified.")
        valid = False
elif(trainval_or_classes == 1):
    if(image_dir == ''):
        print("image_dir must be specified.")
        valid = False
    else:
        train_dir = image_dir
        validation_dir = image_dir

if(valid and trainval_or_classes==0):
    train_datagen = ImageDataGenerator(
          rescale=1./255,
          rotation_range=20,
          width_shift_range=0.2,
          height_shift_range=0.2,
          horizontal_flip=True,
          fill_mode='nearest'
    )

    validation_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
            train_dir,
            target_size=(image_size, image_size),
            batch_size=train_batchsize,
#             class_mode='categorical',
            class_mode='binary'
    )

    validation_generator = validation_datagen.flow_from_directory(
            validation_dir,
            target_size=(image_size, image_size),
            batch_size=val_batchsize,
#             class_mode='categorical',
            class_mode='binary',
            shuffle=False
    )
    
elif(valid and trainval_or_classes==1):
    train_datagen = ImageDataGenerator(
          rescale=1./255,
          rotation_range=20,
          width_shift_range=0.2,
          height_shift_range=0.2,
          horizontal_flip=True,
          fill_mode='nearest',
          validation_split=0.2
    )

#     validation_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
            train_dir,
            target_size=(image_size, image_size),
            batch_size=train_batchsize,
#             class_mode='categorical',
            class_mode='binary',
            subset='training'
    )

    validation_generator = train_datagen.flow_from_directory(
            validation_dir,
            target_size=(image_size, image_size),
            batch_size=val_batchsize,
#             class_mode='categorical',
            class_mode='binary',
            shuffle=False,
            subset='validation'
    )

Found 338 images belonging to 2 classes.
Found 84 images belonging to 2 classes.


In [ ]:
save_path = '/data/ml_data/models'
save_name = 'oilwater'

save = '' # leave this empty
if(save_path == '' or save_name == ''):
    print("save_path and save_name must be specified.")
    valid = False
else:
    save = os.path.join(save_path, save_name) + '.h5'
print("Save path... " + save)

if(valid):
    # Compile the model
    model.compile(
#         loss='categorical_crossentropy',
        loss='binary_crossentropy',
#         optimizer=optimizers.RMSprop(lr=1e-4),
        optimizer=optimizers.Adam(lr=1e-3, decay=1e-3/10),
        metrics=['acc']
    )
    # Train the model
    history = model.fit_generator(
          train_generator,
          steps_per_epoch=train_generator.samples/train_generator.batch_size,
#           steps_per_epoch=5,
#           epochs=30,
          epochs=10,
          validation_data=validation_generator,
          validation_steps=validation_generator.samples/validation_generator.batch_size,
#           validation_steps=2,
          verbose=1
    )

    # Save the model
    model.save(save)

Save path... /data/ml_data/models/oilwater.h5
Epoch 1/10
 9/10 [========================>.....] - ETA: 36s - loss: 10.4068 - acc: 0.3472 

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()

In [ ]:
# Predict
from keras.models import load_model
from keras.preprocessing import image
import numpy as np

img_path = ''

model = load_model('model.h5')
model.compile(
#     loss='categorical_crossentropy',
    loss='binary_corssentropy',
#     optimizer=optimizers.RMSprop(lr=1e-4),
    optimizer=optimizers.Adam(),
    metrics=['acc']
)

img = image.load_img(img_path, target_size=(image_size, image_size))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
images = np.vstack([x])
classes = model.predict_classes(images, batch_size=10)
print(classes)